<a href="https://colab.research.google.com/github/DeepsMoseli/TswanaBert/blob/master/tswanaBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 757kB 9.0MB/s 
     |████████████████████████████████| 1.1MB 26.6MB/s 
     |████████████████████████████████| 3.0MB 54.5MB/s 
     |████████████████████████████████| 890kB 53.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=ba4c03b8f0f346d5d07ee948af3799afd0f44b01517155369938078bd7406797
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:

from pathlib import Path

from tokenizers import ByteLevelBPETokenizer

paths = [str(x) for x in Path("./").glob("**/*.txt")]
paths

['large_test_clean.txt']

## initialize tokenizer and customize training


In [6]:
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=["./large_test_clean.txt"], show_progress=True,vocab_size=25000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])
tokenizer.save_model('.')
print("Trained vocab size: {}".format(tokenizer.get_vocab_size()))

Trained vocab size: 13446


In [7]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = ByteLevelBPETokenizer(
    "./TswanaBert/vocab.json",
    "./TswanaBert/merges.txt",
)

tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)


encoding = tokenizer.encode("gore fa a re merafe yotlhe e nne le.")
print("Encoded String: ", encoding.tokens)

decoded = tokenizer.decode(encoding.ids)
print("Decoded string: {}".format(decoded))

Encoded String:  ['<s>', 'gore', 'Ġfa', 'Ġa', 'Ġre', 'Ġmerafe', 'Ġyotlhe', 'Ġe', 'Ġnne', 'Ġle', '.', '</s>']
Decoded string: <s>gore fa a re merafe yotlhe e nne le.</s>


## Make and Train a language model



In [8]:
!nvidia-smi

Tue Jun 30 20:03:58 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
import torch
torch.cuda.is_available()

True

### Config the model and re-create our tokenizer in transformers


In [10]:
from transformers import RobertaConfig
from transformers import RobertaTokenizerFast

config = RobertaConfig(
    vocab_size=13446,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

tokenizer = RobertaTokenizerFast.from_pretrained("./TswanaBert", max_len=512)

 since we are training from scratch, we initialize from config not a checkpoint of saved model


In [11]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config = config)
print("Number of parameters: ", model.num_parameters())

Number of parameters:  54446982


## Make and register training dataset

In [12]:
%%time
from transformers import LineByLineTextDataset
from transformers import DataCollatorForLanguageModeling

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./large_test_clean.txt",
    block_size=128,
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

CPU times: user 863 ms, sys: 15.9 ms, total: 878 ms
Wall time: 541 ms


Initialize model

In [13]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./TswanaBert",
    overwrite_output_dir=True,
    num_train_epochs=200,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)

In [14]:
%%time
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)




CPU times: user 2h 20s, sys: 1h 22min 41s, total: 3h 23min 2s
Wall time: 3h 25min 42s


TrainOutput(global_step=31800, training_loss=2.6185353006338175)

In [25]:
trainer.save_model("./TswanaBert")
tokenizer.save_pretrained("./TswanaBert")

('./TswanaBert/vocab.json',
 './TswanaBert/merges.txt',
 './TswanaBert/special_tokens_map.json',
 './TswanaBert/added_tokens.json')

-----
# Now lets Test

In [26]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./TswanaBert",
    tokenizer="./TswanaBert"
)

fill_mask("Ntshopotse tsela e <mask>.")

/usr/local/lib/python3.6/dist-packages/transformers/modeling_auto.py:791: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


[{'score': 0.14202992618083954,
  'sequence': '<s>Ntshopotse tsela e tletse.</s>',
  'token': 1774,
  'token_str': 'Ġtletse'},
 {'score': 0.0671372264623642,
  'sequence': '<s>Ntshopotse tsela e tsweletse.</s>',
  'token': 705,
  'token_str': 'Ġtsweletse'},
 {'score': 0.058057453483343124,
  'sequence': '<s>Ntshopotse tsela e dirwa.</s>',
  'token': 958,
  'token_str': 'Ġdirwa'},
 {'score': 0.03353468328714371,
  'sequence': '<s>Ntshopotse tsela e godile.</s>',
  'token': 2063,
  'token_str': 'Ġgodile'},
 {'score': 0.019653351977467537,
  'sequence': '<s>Ntshopotse tsela e tsile.</s>',
  'token': 1291,
  'token_str': 'Ġtsile'}]

## save model on huggingface site

In [ ]:
from transformers import TFRobertaForMaskedLM

tf_model = TFRobertaForMaskedLM.from_pretrained("./TswanaBert", from_pt=True)
tf_model.save_pretrained("./TswanaBert")

## Data sources

The dataset I train on was collected from the following sites.

* http://setswana.blogspot.com/
* https://omniglot.com/writing/tswana.php
* http://www.dailynews.gov.bw/
* http://www.mmegi.bw/index.php
* https://tsena.co.bw
* http://www.botswana.co.za/Cultural_Issues-travel/botswana-country-guide-en-route.html
*



In [28]:
!transformers-cli login

2020-07-01 00:13:26.777193: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1

        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        
Username: MoseliMotsoehli
Password: 
Login successful
Your token: uILpqJCPboRgIqtOZcthQNyjnJUTpVVobIurrCjjfTrkEBXuGvmIXCUjPbnokBIFNfffBpdBoSazxNEvYyjBzGeKUURKzwASyXkDOZcKunAiMnMwZKSFKjvJbWtACXCy 

Your token has been saved to /root/.huggingface/token


In [30]:
!transformers-cli upload ./TswanaBert/

2020-07-01 00:16:24.054267: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
About to upload file /content/TswanaBert/config.json to S3 under filename TswanaBert/config.json and namespace MoseliMotsoehli
About to upload file /content/TswanaBert/tf_model.h5 to S3 under filename TswanaBert/tf_model.h5 and namespace MoseliMotsoehli
About to upload file /content/TswanaBert/special_tokens_map.json to S3 under filename TswanaBert/special_tokens_map.json and namespace MoseliMotsoehli
About to upload file /content/TswanaBert/vocab.json to S3 under filename TswanaBert/vocab.json and namespace MoseliMotsoehli
About to upload file /content/TswanaBert/merges.txt to S3 under filename TswanaBert/merges.txt and namespace MoseliMotsoehli
About to upload file /content/TswanaBert/pytorch_model.bin to S3 under filename TswanaBert/pytorch_model.bin and namespace MoseliMotsoehli
About to upload file /content/TswanaBert/tokenizer_config.js